In [1]:
import json
import torch
import gpytorch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error

#### Train the GPmodel with best hyperparameters

In [9]:
# Load the data
df = pd.read_csv('data/df_cleaned_10292024.csv')

# Separate df into training and testing data
df_train = df[df['ground_truth'].notnull()]
df_test = df[df['ground_truth'].isnull()]

# Shuffle the data
df_train = df_train.sample(frac=1, random_state=42)
df_train = df_train.reset_index(drop=True)

In [5]:
# Define the Latent Embedding Kernel and combine with other features
class LatentEmbeddingAndFeatureKernel(gpytorch.kernels.Kernel):
    def __init__(self, num_areas, embedding_dim, feature_kernel, **kwargs):
        super(LatentEmbeddingAndFeatureKernel, self).__init__(**kwargs)
        
        # Learnable embeddings for each area (discrete)
        self.embedding = torch.nn.Embedding(num_areas, embedding_dim)
        
        # Kernel for continuous features
        self.feature_kernel = feature_kernel
    
    def forward(self, x1, x2, diag=False, **params):
        # Split x1 and x2 into bboxid and continuous features
        bboxid_x1, features_x1 = x1[:, 0].long(), x1[:, 1:]
        bboxid_x2, features_x2 = x2[:, 0].long(), x2[:, 1:]

        # Compute the embeddings for bboxid
        embed_x1 = self.embedding(bboxid_x1)
        embed_x2 = self.embedding(bboxid_x2)

        # Standardize the continuous features
        features_x1 = (features_x1 - features_x1.mean(dim=0)) / features_x1.std(dim=0)
        features_x2 = (features_x2 - features_x2.mean(dim=0)) / features_x2.std(dim=0)

        # Combine embeddings with the continuous features
        combined_x1 = torch.cat([embed_x1, features_x1], dim=-1)
        combined_x2 = torch.cat([embed_x2, features_x2], dim=-1)

        # Apply the RBF kernel to the combined inputs
        if diag:
            # Return only the diagonal elements when requested
            return self.feature_kernel(combined_x1, combined_x2, diag=True)
        else:
            # Return the full covariance matrix when diagonal is not requested
            return self.feature_kernel(combined_x1, combined_x2)

    def diag(self, x):
        # Handle diagonal requests by passing `diag=True` to the forward method
        return self.forward(x, x, diag=True)
        
        # # Apply a standard kernel to the combined inputs (embeddings + continuous features)
        # return self.feature_kernel(combined_x1, combined_x2)

# Define the GP Model with Poisson Likelihood
class GPModel(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points, num_areas):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = gpytorch.variational.VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPModel, self).__init__(variational_strategy)

        # Define the kernel: latent embedding for area codes + kernel for temperature and demographic features
        feature_kernel = gpytorch.kernels.RBFKernel(ard_num_dims=embedding_dim + 7) # 7  additional features
        self.covar_module = LatentEmbeddingAndFeatureKernel(num_areas, embedding_dim, feature_kernel)
        self.mean_module = gpytorch.means.ConstantMean()

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    
# Define the Poisson likelihood for count data (number of tents)
class PoissonLikelihood(gpytorch.likelihoods.Likelihood):
    def forward(self, function_samples, *args, **kwargs):
        # Apply the log-link function (exponentiate the latent GP)
        return torch.distributions.Poisson(rate=function_samples.exp())

In [10]:
# Train the model with best hyperparameters
embedding_dim = 3
batch_size = 1000
num_inducing_points = 500
learning_rate = 0.01

# Get the number of unique areas and days
num_areas = df_train['bboxid'].nunique()
num_days = df_train['timestamp'].nunique()

# Create a bboxid tensor
bboxid_tensor = torch.tensor(df_train['bboxid'].astype('category').cat.codes.values, dtype=torch.long)

# Create a temperature tensor
temperature_tensor = torch.tensor(df_train[['max','min','precipitation']].values, dtype=torch.float32)

# Create a demographic tensor
demographic_tensor = torch.tensor(df_train[['total_population','white_ratio','black_ratio','hh_median_income']].values, dtype=torch.float32)

# Create a ground truth tensor
target_tensor = torch.tensor(df_train['ground_truth'].values, dtype=torch.float32)

# Prepare the dataset
train_dataset = TensorDataset(torch.cat([bboxid_tensor.unsqueeze(-1), temperature_tensor, demographic_tensor], dim=-1), target_tensor)

# Set up a DataLoader for mini-batching
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [19]:
print(bboxid_tensor.shape)
print(temperature_tensor.shape)
print(demographic_tensor.shape)
print(target_tensor.shape)

torch.Size([263883])
torch.Size([263883, 3])
torch.Size([263883, 4])
torch.Size([263883])


In [23]:
df_train['bboxid'].nunique()

4377

In [21]:
len(bboxid_tensor.unique())

4377

In [15]:
len(train_dataset)

263883

In [24]:
# Concatenate the bboxid tensor, temperature tensor, and demographic tensor into a single tensor
train_x = torch.cat([bboxid_tensor.unsqueeze(-1), temperature_tensor, demographic_tensor], dim=-1)

# Define inducing points for the sparse approximation (subset of training data)
# Randomly select inducing points
inducing_points = train_x[torch.randperm(train_x.size(0))[:num_inducing_points]] 

# Initialize model and likelihood
model = GPModel(inducing_points, num_areas=num_areas)
#model = model.to(device)
likelihood = PoissonLikelihood()
#likelihood = likelihood.to(device)

# Training the model
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {'params': model.parameters()},
    {'params': likelihood.parameters()},
], lr=learning_rate)

# Loss function: Variational ELBO
mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=len(train_dataset))

# Training loop
training_iterations = 100
for j in range(training_iterations):
    model.train()
    likelihood.train()

    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        output = model(batch_x)
        loss = -mll(output, batch_y)
        loss.backward()
        optimizer.step()

    if j % 10 == 0:
        print(f'Iteration {j}/{training_iterations} - Loss: {loss.item()}')

    # Save the model
checkpoint = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss.item(),
}
torch.save(checkpoint, f'checkpoints/checkpoint_indpts_{num_inducing_points}_embdim_{embedding_dim}.pth')
print("Model saved successfully!")

Iteration 0/100 - Loss: 1.0771100521087646
Iteration 10/100 - Loss: 1.0547429323196411
Iteration 20/100 - Loss: 1.0673983097076416
Iteration 30/100 - Loss: 1.0710989236831665
Iteration 40/100 - Loss: 1.0386900901794434
Iteration 50/100 - Loss: 1.043838620185852
Iteration 60/100 - Loss: 1.0705124139785767
Iteration 70/100 - Loss: 1.0593180656433105
Iteration 80/100 - Loss: 1.0335173606872559
Iteration 90/100 - Loss: 1.0617444515228271
Model saved successfully!


In [25]:
print(model.state_dict())

OrderedDict([('variational_strategy.inducing_points', tensor([[ 3.7840e+03,  5.9684e+01,  5.1546e+01,  ...,  3.3925e-01,
         -8.0264e-01,  1.6471e+05],
        [ 3.4690e+03,  6.0146e+01,  4.9315e+01,  ...,  7.8183e-01,
         -1.1185e+00,  6.5694e+04],
        [ 3.9160e+03,  7.2310e+01,  5.5366e+01,  ...,  1.0520e+00,
         -1.5606e+00,  1.2900e+05],
        ...,
        [ 3.0410e+03,  5.7748e+01,  5.6745e+01,  ..., -2.0596e+00,
          1.3280e+00,  1.2152e+05],
        [ 2.6300e+03,  6.4484e+01,  5.2703e+01,  ...,  1.1623e+00,
         -5.5317e-01,  1.4321e+05],
        [ 2.8070e+03,  5.3306e+01,  4.8291e+01,  ...,  2.3263e+00,
         -2.5910e-01,  9.3438e+04]])), ('variational_strategy.variational_params_initialized', tensor(1)), ('variational_strategy.updated_strategy', tensor(True)), ('variational_strategy._variational_distribution.variational_mean', tensor([ 1.1589e+00,  1.5812e-01,  2.5162e-01,  2.8896e-01,  3.3758e-01,
         2.9814e-01,  4.5029e-01, -2.7556e-02,

In [26]:
# Load the model and optimizer for prediction
# Initialize model and likelihood
model = GPModel(inducing_points, num_areas=num_areas)
likelihood = PoissonLikelihood()
optimizer = torch.optim.Adam([
    {'params': model.parameters()},
    {'params': likelihood.parameters()},
], lr=learning_rate)

# Load the checkpoint
checkpoint = torch.load('checkpoints/checkpoint_indpts_500_embdim_3.pth')

# Restore model and optimizer states
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [27]:
# Evaluate the model
model.eval()
likelihood.eval()


PoissonLikelihood()

In [1]:
print(df_train['bboxid'].nunique())
print(df_test['bboxid'].nunique())

NameError: name 'df_train' is not defined

In [32]:
# Predictions for Test set
# Create a bboxid tensor
bboxid_tensor_test = torch.tensor(df_test['bboxid'].astype('category').cat.codes.values, dtype=torch.long)

# Create a temperature tensor
temperature_tensor_test = torch.tensor(df_test[['max','min','precipitation']].values, dtype=torch.float32)

# Create a demographic tensor
demographic_tensor_test = torch.tensor(df_test[['total_population','white_ratio','black_ratio','hh_median_income']].values, dtype=torch.float32)

# Concatenate the bboxid tensor, temperature tensor, and demographic tensor into a single tensor
test_dataset = TensorDataset(torch.cat([bboxid_tensor_test.unsqueeze(-1), temperature_tensor_test, demographic_tensor_test], dim=-1)) 
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Concatenate the bboxid tensor, temperature tensor, and demographic tensor into a single tensor
test_x = torch.cat([bboxid_tensor_test.unsqueeze(-1), temperature_tensor_test, demographic_tensor_test], dim=-1)

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    predictions = likelihood(model(test_x))

# with torch.no_grad(), gpytorch.settings.fast_pred_var():
#     predictions = []
#     for batch_x in test_loader:
#         batch_x = batch_x[0]
#         output = likelihood(model(batch_x))
#         predictions.append(output.mean.detach().cpu().numpy())